# LDA 统计模型对新闻进行主题分析

在LDA模型中，一篇文档生成的方式如下：
1. 从狄利克雷分布中取样生成文档 i 的主题分布
2. 从主题的多项式分布中取样生成文档i第 j 个词的主题
3. 从狄利克雷分布中取样生成主题对应的词语分布
4. 从词语的多项式分布中采样最终生成词语

In [40]:
# 新闻爬取xlxs文件目录：
NEWS_PATH = './news/'
#语料库文件路径：
DATA_PATH = './data/news_words.txt'

#coding=utf-8  
import codecs  
from gensim import corpora  
from gensim.models import LdaModel  
from gensim.corpora import Dictionary  
fr=open(DATA_PATH,'r')  
train=[]  
for line in fr.readlines():  
    line=line.split(' ')  
    train.append(line)  
print(len(train))
print(' '.join(train[2]))
  
dictionary = corpora.Dictionary(train)  
corpus = [ dictionary.doc2bow(text) for text in train ]  
print("Loading corpus finished")

/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


42951
全球 对外 开放 对外开放 再添 金色 动力 中国 最新 公布 对外 外贸 贸易 对外贸易 数据 放缓 迹象 月初 第九 九次 第九次 金砖 国家 领导 领导人 厦门 会晤 希望 中国 公布 最新 对外 外贸 贸易 对外贸易 数据 显示 前 月 中国 进出 出口 进出口 总值 同比 增长 出口 出口值 增长 进口 值 增长 月 进出 出口 进出口 值 出口 出口值 进口 值 同比 增长 总体 增速 今年以来 月 外 单月 出口 出口值 同比 增速 再度 滑落 个位 位数 个位数 究其 原因 究其原因 出口 商品 出口商 出口商品 一方 纺织 服装 传统 优势 劳动 密集 密集型 劳动密集型 产品 出口 下降 一方 另一方 钢材 铝材 化肥 煤 焦炭 两 高一 资 产品 出口 下降 两类 产品 出口 下降 拉低 月 当月 出口 增速 约 百分 分点 百分点 贸易 伙伴 月 当月 中国 欧美 日 发达 经济 经济体 出口 增速 仅为 高于 略高于 同期 总体 出口 增速 罗斯 俄罗斯 巴西 南非 印度 新兴 经济 经济体 出口 增速 达 高出 总体 出口 增速 4.4 百分 分点 百分点 月初 全球 目光 继 去年 月 杭州 G20 峰会 月 北京 一带 一路 一带一路 高峰 论坛 高峰论坛 再度 投向 中国 聚焦 中国 厦门 第九 九次 第九次 金砖 国家 领导 领导人 会晤 成功 落幕 2006 年 金砖 国家 合作 机制 正式 建立 世界 贸易 组织 世界贸易组织 数据 金砖 国家 经济 总量 经济总量 占 全球 经济 总量 经济总量 比重 对外 外贸 贸易 对外贸易 总值 占 全球 贸易 总值 比重 此刻 十年 发展 期间 遭遇 全球 金融 危机 金融危机 地缘 政治 事件 影响 金砖 国家 全球 经贸 发展 中 地位 与日俱增 经济 总量 经济总量 占 全球 比重 增加 全球 经济 增长 贡献 贡献率 超过 对外 外贸 贸易 对外贸易 总值 占 同期 全球 贸易 总值 比重 全球 外贸 增速 贡献 贡献率 超过 值得 值得一提的是 十年 金砖 国家 中国 对外 外贸 贸易 对外贸易 领域 中 显示 强劲 增长 动力 紧密 合作 交往 数据 显示 2016 年 中国 罗斯 俄罗斯 巴西 南非 印度 等国 外贸 进出 出口 进出口 值 十年 年间 十年

In [41]:
#训练LDA模型
lda = LdaModel(corpus=corpus, id2word=dictionary, num_topics=200)  


/home/inesa/.local/lib/python3.5/site-packages/gensim/models/ldamodel.py:775: RuntimeWarning: divide by zero encountered in log
  diff = np.log(self.expElogbeta)


In [45]:

lda.save('./model/lda.model')


In [46]:
lda = LdaModel.load('./model/lda.model')

In [17]:

topic_list=lda.print_topics(20)  
print(type(lda.print_topics(20)))
print(len(lda.print_topics(20)))
  
for topic in topic_list:  
    print(topic) 
print("第一主题")
print(lda.print_topic(1))

<class 'list'>
20
(32, '0.110*"万达" + 0.029*"财经" + 0.028*"绿色" + 0.026*"建筑" + 0.024*"二维" + 0.023*"客户端" + 0.022*"二维码" + 0.018*"商店" + 0.018*"第一" + 0.018*"搜索"')
(14, '0.055*"试验" + 0.046*"跨境" + 0.035*"自贸" + 0.031*"试验区" + 0.030*"投资" + 0.017*"境外" + 0.015*"人民币" + 0.013*"区内" + 0.012*"台湾" + 0.011*"贸易"')
(194, '0.089*"山东" + 0.058*"江西" + 0.054*"辽宁" + 0.048*"安徽" + 0.045*"省份" + 0.041*"福建" + 0.026*"江苏" + 0.024*"山东省" + 0.019*"陕西" + 0.017*"南昌"')
(188, '0.017*"改革" + 0.016*"发展" + 0.014*"工作" + 0.013*"中央" + 0.010*"社会" + 0.010*"建设" + 0.010*"制度" + 0.009*"推进" + 0.007*"会议" + 0.007*"领导"')
(5, '0.079*"舟山" + 0.024*"拳头" + 0.023*"士兵" + 0.016*"深中" + 0.015*"拳头产品" + 0.014*"体育赛事" + 0.013*"每笔" + 0.013*"Q2" + 0.012*"行政区划" + 0.011*"首列"')
(107, '0.013*"家族企业" + 0.011*"伤员" + 0.009*"镕" + 0.008*"内情" + 0.008*"李在" + 0.007*"参阅" + 0.007*"黄颂平" + 0.006*"刚过去" + 0.005*"退为" + 0.005*"以退为进"')
(83, '0.063*"手续" + 0.052*"续费" + 0.050*"手续费" + 0.034*"开户" + 0.023*"财经新闻" + 0.021*"中央组织部" + 0.021*"火星" + 0.020*"万分" + 0.020*"购买" + 0.014*"万分之"')
(52, 

In [10]:

  
print('给定一个新文档，输出其主题分布' )
  
#test_doc = list(new_doc) #新文档进行分词  
test_doc=train[24537]#查看训练集中第三个样本的主题分布  
print("".join(test_doc))
doc_bow = dictionary.doc2bow(test_doc)      #文档转换成bow  
doc_lda = lda[doc_bow]                   #得到新文档的主题分布  
#输出新文档的主题分布  
print(doc_lda)  
for topic in doc_lda:  
    print("%s\t%f\n"%(lda.print_topic(topic[0]), topic[1]) )

给定一个新文档，输出其主题分布
回归A股方案拆解周鸿祎控权有方天价借壳闯关回归A股尘埃落定尘埃落定回归方案江南嘉捷午夜公告浮出水面月日分钟江南嘉捷突发条公告此前借壳绯闻坐实公告显示江南嘉捷拟出售资产负债资产置换发行股份购买六零科技股份科技股科技股份有限公司有限公司简称股权股权亿元借壳周鸿祎合计控制江南嘉捷股份红衣教主公司控制控制权借壳方案当于相当于年IPO募资总额总额近四成刷新A股借壳金额重量量级重量级借壳草案平衡各方利益面临监管拷问过会难度备受关注日下午证监证监会借壳做出回应一方支持境外市场上市优质中资企业中资企业回到境内市场并购重组称高度关注严厉打击严厉打击涉及内幕交易违法违规违规行为周鸿祎控制术选择江南嘉捷外界普遍认为看中体量小好控制股份稀释少股东利益利于有利于周鸿祎借壳控制借壳草案披露周鸿祎奇信志成处于控股低位两家公司股东数量众多层层穿透实际上单个股东持股数量超过家架构保证周鸿祎持股比例优势间接方式控制股份数草案披露借壳周鸿祎合计控制江南嘉捷股份处于控制地位草案披露借壳交易分为两大部大部分资产出售江南嘉捷分两步第一一步第一步先公司子公司嘉捷机电资产负债业务打包划转至嘉捷机电嘉捷机电股权现金转让金志峰金祖铭父子江南嘉捷控制第三三方第三方交易作价亿元二步第二步将嘉捷机电股权股东拥有股权等值置换股东股权转让金志峰金祖铭指定第三三方第三方上海一位投行负责责人负责人第一财经分析称分两步两步走是因为股东分散控制成本维护股东权益股东权益做出设计资产置换发行股份购买资产股权亿元作价扣除嘉捷机电股权作价亿元置换外差额亿元江南嘉捷发行股份方式股东购买发行发行价元股发行规模约亿股交易奇信志成持有江南嘉捷总股本控股股东持有江南嘉捷股份奇信志成天津众信间接控江南嘉捷股份合计控制江南嘉捷股份江南嘉捷控制金氏父子合计持股降为周鸿祎控制江南嘉捷股份持有奇信志成股份第一股东担任奇信志成执行董事奇信志成执行董事选举需由周鸿祎提名周鸿祎有权提名新董事人选取代提名执行董事股东股东会决议事项中需奇信志成行使表决表决权事项奇信志成股东同意应周鸿祎指示表决重组实施完毕股东给出年年年三年内扣非净利利润净利润低于亿元亿元亿元业绩承诺合计高达亿元增长增长率年增长率江南嘉捷财务数据财务数据截止年中报资产总资产亿元总负债亿元负债负债率约年中报净利利润净利润亿元亿元亿元壳质量高江南嘉捷相比体量营收显得庞然庞然大物年

In [2]:
train[24537]

NameError: name 'train' is not defined

## 标记数据导入与分类训练

标记数据来源： 第一财经新闻
标记格式：新闻标题/正文/关键词
利用LDA的模型，将标题+正文作为字符串，对标记数据进行向量化，标签为关键词。




In [19]:
import jieba
import csv
import numpy as np  
import os  
import pandas as pd


LABEL_PATH = './labels/'

LABEL_CSV = './data/labels.csv'


# 加载停用词，输入停用词文件，输出停用词list
def stopwordslist(filepath):  
    stopwords = [line.strip() for line in open(filepath, 'r', encoding='utf-8').readlines()]  
    return stopwords

temp_stop_list = ['\u3000','\xa0','\t']
stop_words = stopwordslist("./utils/stopwords.txt") + temp_stop_list
# 为结巴分词词库加载股票名词汇
jieba.load_userdict('./data/user_dict.txt')

 
## jieba分词：输入string & 停用词文件，输出分词结果list
def jieba_split(content):
    '''
    content: 输入文本（string）
    stop_path: 停用词字典文件路径（string）
    返回：list，jieba分词结果
    '''
    str_content = str(content).replace('\t', '').replace('\n', '').replace(' ','')
    str_words = ','.join(jieba.cut_for_search(str_content)).split(",")
    ret_list = []
    for word in str_words:
        if word not in stop_words:
            if word[-1] != '%':
                ret_list.append(word)
    return ret_list


    

In [48]:
# Naive Bayes

# turn lda result into list
def lda2list(lda,topic_n):
    lda_dict = dict(lda)
    lda_list = [0] * topic_n
    for i in range(topic_n):
        lda_list[i] = lda_dict.get(i,0)
    return lda_list

#df_label = pd.DataFrame(columns=['words','label'])  
news_vec = []
news_labels = []


files = os.listdir(LABEL_PATH)
print(files)
for fname in files:
    fpath = LABEL_PATH + fname
    if 'csv' in fpath:
        print(fpath)
        file_data = pd.read_csv(fpath)
        file_data.rename(columns={'标题':'title', '正文':'content','正文1':'content',"字段1_文本":"title","关键词":"plate"}, inplace = True)
        for index, row in file_data.iterrows():
        #print(row.content)
            news_word = jieba_split(str(row.title) + str(row.content))
            news_bow = dictionary.doc2bow(news_word)      #文档转换成bow  
            news_lda = lda2list(lda[news_bow],201) #得到lda向量
            news_vec.append(news_lda)
            news_labels.append(row.plate)
    print("done")

            

['第一财经板块对应新闻-0423-2.csv', '第一财经板块对应新闻-0423-1.csv', '第一财经板块对应新闻-0419-2.csv', '第一财经板块对应新闻-0413.csv', '第一财经板块对应新闻.csv', '第一财经板块对应新闻-0419-1.csv', '第一财经板块对应新闻-0416.csv']
./labels/第一财经板块对应新闻-0423-2.csv
done
./labels/第一财经板块对应新闻-0423-1.csv
done
./labels/第一财经板块对应新闻-0419-2.csv
done
./labels/第一财经板块对应新闻-0413.csv
done
./labels/第一财经板块对应新闻.csv
done
./labels/第一财经板块对应新闻-0419-1.csv
done
./labels/第一财经板块对应新闻-0416.csv
done


In [49]:
print(len(news_labels))

55793


In [56]:
#高斯朴素贝叶斯  
X=np.array([np.array(xi) for xi in news_vec])
Y = np.array(news_labels)
print(type(X[0]))
print(type(Y))
print(len(X))
#X = np.array([[-1, -1], [-2, -1], [-3, -2], [1, 1], [2, 1], [3, 2]])  
#Y = np.array([1, 1, 1, 2, 2, 2])  
from sklearn.naive_bayes import GaussianNB  
clf = GaussianNB().fit(X, Y)  
#print(clf.predict([[-0.8,-1]]))
  
''''' 
partial_fit说明：增量的训练一批样本 
这种方法被称为连续几次在不同的数据集，从而实现核心和在线学习，这是特别有用的，当数据集很大的时候，不适合在内存中运算 
该方法具有一定的性能和数值稳定性的开销，因此最好是作用在尽可能大的数据块（只要符合内存的预算开销） 
'''  
#clf_pf = GaussianNB().partial_fit(X, Y, np.unique(Y))  


<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
55793


"'' \npartial_fit说明：增量的训练一批样本 \n这种方法被称为连续几次在不同的数据集，从而实现核心和在线学习，这是特别有用的，当数据集很大的时候，不适合在内存中运算 \n该方法具有一定的性能和数值稳定性的开销，因此最好是作用在尽可能大的数据块（只要符合内存的预算开销） \n"

In [62]:
# coding=gbk  

   
import time    
from sklearn import metrics    
import pickle as pickle    
import pandas as pd  
  
    
# Multinomial Naive Bayes Classifier    
def naive_bayes_classifier(train_x, train_y):    
    from sklearn.naive_bayes import MultinomialNB    
    model = MultinomialNB(alpha=0.01)    
    model.fit(train_x, train_y)    
    return model    
    
    
# KNN Classifier    
def knn_classifier(train_x, train_y):    
    from sklearn.neighbors import KNeighborsClassifier    
    model = KNeighborsClassifier()    
    model.fit(train_x, train_y)    
    return model    
    
    
# Logistic Regression Classifier    
def logistic_regression_classifier(train_x, train_y):    
    from sklearn.linear_model import LogisticRegression    
    model = LogisticRegression(penalty='l2')    
    model.fit(train_x, train_y)    
    return model    
    
    
# Random Forest Classifier    
def random_forest_classifier(train_x, train_y):    
    from sklearn.ensemble import RandomForestClassifier    
    model = RandomForestClassifier(n_estimators=8)    
    model.fit(train_x, train_y)    
    return model    
    
    
# Decision Tree Classifier    
def decision_tree_classifier(train_x, train_y):    
    from sklearn import tree    
    model = tree.DecisionTreeClassifier()    
    model.fit(train_x, train_y)    
    return model    
    
    
# GBDT(Gradient Boosting Decision Tree) Classifier    
def gradient_boosting_classifier(train_x, train_y):    
    from sklearn.ensemble import GradientBoostingClassifier    
    model = GradientBoostingClassifier(n_estimators=200)    
    model.fit(train_x, train_y)    
    return model    
    
    
# SVM Classifier    
def svm_classifier(train_x, train_y):    
    from sklearn.svm import SVC    
    model = SVC(kernel='rbf', probability=True)    
    model.fit(train_x, train_y)    
    return model    
    
# SVM Classifier using cross validation    
def svm_cross_validation(train_x, train_y):    
    from sklearn.grid_search import GridSearchCV    
    from sklearn.svm import SVC    
    model = SVC(kernel='rbf', probability=True)    
    param_grid = {'C': [1e-3, 1e-2, 1e-1, 1, 10, 100, 1000], 'gamma': [0.001, 0.0001]}    
    grid_search = GridSearchCV(model, param_grid, n_jobs = 1, verbose=1)    
    grid_search.fit(train_x, train_y)    
    best_parameters = grid_search.best_estimator_.get_params()    
    for para, val in list(best_parameters.items()):    
        print(para, val)    
    model = SVC(kernel='rbf', C=best_parameters['C'], gamma=best_parameters['gamma'], probability=True)    
    model.fit(train_x, train_y)    
    return model    
    
def read_data(data_file):    
    data = pd.read_csv(data_file)  
    train = data[:int(len(data)*0.9)]  
    test = data[int(len(data)*0.9):]  
    train_y = train.label  
    train_x = train.drop('label', axis=1)  
    test_y = test.label  
    test_x = test.drop('label', axis=1)  
    return train_x, train_y, test_x, test_y  
        

/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [67]:
import random
X = np.array([np.array(xi) for xi in news_vec])
Y = np.array(news_labels)

test_classifiers = ['NB', 'KNN', 'LR', 'RF', 'DT', 'SVM','SVMCV', 'GBDT']    
classifiers = {'NB':naive_bayes_classifier,     
              'KNN':knn_classifier,    
               'LR':logistic_regression_classifier,    
               'RF':random_forest_classifier,    
               'DT':decision_tree_classifier,    
              'SVM':svm_classifier,    
            'SVMCV':svm_cross_validation,    
             'GBDT':gradient_boosting_classifier    
}    

def test_classifier(classifier):
    print("********classifier: ",classifier,"***********")
    start_time = time.time()    
    model = classifiers[classifier](X, Y)
    print('training took %fs!' % (time.time() - start_time)) 
    for i in range(100):
        n = random.randrange(1,len(X))
        print(model.predict([X[n]]))
        print(Y[n])

In [68]:
test_classifier("KNN")

********classifier:  KNN ***********


/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


training took 15.636353s!
['创投']
创投
['互联网彩票']
金融IC
['军民融合']
车联网
['农机']
太阳能
['PM2.5']
金改
['上海自贸区']
福建自贸区
['举牌']
共享单车
['迪士尼']
迪士尼
['互联网医疗']
互联网医疗
['上海自贸区']
上海自贸区
['共享单车']
太阳能
['基因测序']
节能环保
['军工']
黄金
['京津冀一体化']
金改
['PM2.5']
风电
['广东自贸区']
广东自贸区
['互联网金融']
航运
['农机']
钛白粉
['基因测序']
基因测序
['军工']
军工
['区块链']
锂电池
['农村电商']
车联网
['阿里巴巴']
阿里巴巴
['供应链金融']
供应链金融
['电子信息']
量子通信
['基因测序']
航运
['电子竞技']
白酒
['啤酒']
大数据
['大飞机']
太阳能
['军工']
军工
['举牌']
举牌
['PM2.5']
PM2.5
['人脸识别']
迪士尼
['人工智能']
黄金
['阿里巴巴']
手机游戏
['深港通']
深港通
['核电']
核电
['互联网金融']
互联网金融
['阿里巴巴']
阿里巴巴
['人脸识别']
互联网金融
['工业4.0']
工业4.0
['电子商务']
广东自贸区
['锂电池']
锂电池
['农业现代化']
风电
['供应链金融']
供应链金融
['跨境电商']
跨境电商
['锂电池']
锂电池
['互联网+']
生物医药
['高端装备']
煤化工
['共享单车']
区块链
['互联网+']
军民融合
['上海自贸区']
汽车电子
['迪士尼']
迪士尼
['啤酒']
啤酒
['石墨烯']
石墨烯
['上海自贸区']
上海自贸区
['高送转']
节能环保
['广东自贸区']
广东自贸区
['OLED']
水泥
['PM2.5']
PM2.5
['高端装备']
高端装备
['互联网金融']
工业4.0
['电子信息']
跨境电商
['核电']
核电
['农业现代化']
美丽中国
['PM2.5']
PM2.5
['安防']
大数据
['共享单车']
钛白粉
['区块链']
区块链
['阿里巴巴']
人工智能
['ST板块']
风电
['迪士尼']
区块链
['工业4.0']
工业4.0
['啤酒'

In [69]:
test_classifier("RF")

********classifier:  RF ***********


/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


training took 5.989762s!
['高端装备']
高端装备
['水泥']
水泥
['融资融券']
融资融券
['白酒']
白酒
['创投']
创投
['黄金']
黄金
['风电']
风电
['生态农业']
生态农业
['海工装备']
海工装备
['航运']
航运
['建筑节能']
建筑节能
['生物质能']
生物质能
['量子通信']
量子通信
['乳业']
黄金
['风电']
风电
['能源互联网']
能源互联网
['乳业']
乳业
['电子发票']
电子发票
['量子通信']
量子通信
['军民融合']
高校
['集成电路']
集成电路
['白马股']
白马股
['期货概念']
期货概念
['冷链物流']
冷链物流
['阿里巴巴']
阿里巴巴
['水利']
水利
['军民融合']
军民融合
['农业现代化']
农业现代化
['阿里巴巴']
阿里巴巴
['食品安全']
食品安全
['高端装备']
高端装备
['基因测序']
基因测序
['区块链']
区块链
['充电桩']
充电桩
['太阳能']
太阳能
['举牌']
举牌
['基因测序']
基因测序
['农机']
农机
['大飞机']
大飞机
['共享单车']
共享单车
['阿里巴巴']
阿里巴巴
['太阳能']
太阳能
['航运']
航运
['生物医药']
生物医药
['工业4.0']
工业4.0
['电子商务']
电子商务
['工业4.0']
工业4.0
['大飞机']
大飞机
['核电']
核电
['大数据']
大数据
['互联网+']
互联网+
['创投']
创投
['农村电商']
农村电商
['创投']
创投
['食品安全']
食品安全
['农村电商']
农村电商
['阿里巴巴']
阿里巴巴
['基因测序']
基因测序
['美丽中国']
美丽中国
['高铁']
高铁
['水泥']
高校
['钛白粉']
钛白粉
['车联网']
车联网
['水泥']
水泥
['太阳能']
太阳能
['人脸识别']
人脸识别
['白酒']
白酒
['军民融合']
军民融合
['互联网金融']
互联网金融
['农机']
农机
['高铁']
高铁
['量子通信']
量子通信
['融资融券']
融资融券
['深港通']
深港通
['乳业']
乳业
['迪士尼']
黄金
['区块链']
区块链
['高送转']
高送

In [51]:
X = np.array([[-1, -1], [-2, -1], [-3, -2], [1, 1], [2, 1], [3, 2]])
type(X[0])

numpy.ndarray